<table align="left"><td><a target="_blank" href="https://colab.research.google.com/github/superannotateai/model-deployment-tutorials/blob/main/JETSON/SuperAnnotate_JETSON_Deeplabv3%2B_Deployment.ipynb"><img src="https://user-images.githubusercontent.com/25985824/104791629-6e618700-5769-11eb-857f-d176b37d2496.png" height="32" width="32"> Try in Google Colab</a></td></table>

# 1. Install Prerequisites 
(Please click on **RESTART RUNTIME** button when it appears in the output of this code block)



In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')


In [ ]:
!cp /content/drive/MyDrive/TensorRT-7.2.3.4.Ubuntu-18.04.x86_64-gnu.cuda-11.0.cudnn8.1.tar.gz /content/
!cp /content/drive/MyDrive/cudnn-11.2-linux-x64-v8.1.1.33.tgz /content/

!tar -xzvf cudnn-11.2-linux-x64-v8.1.1.33.tgz

!sudo cp cuda/include/cudnn*.h /usr/local/cuda/include 
!sudo cp -P cuda/lib64/libcudnn* /usr/local/cuda/lib64 
!sudo chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*

!tar xzvf TensorRT-7.2.3.4.Ubuntu-18.04.x86_64-gnu.cuda-11.0.cudnn8.1.tar.gz
os.environ['LD_LIBRARY_PATH'] += ':/content/TensorRT-7.2.3.4/lib/'

%cd /content/TensorRT-7.2.3.4/python/
!sudo pip3 install tensorrt-7.2.3.4-cp37-none-linux_x86_64.whl

%cd /content/TensorRT-7.2.3.4/uff/
!sudo pip3 install uff-0.6.9-py2.py3-none-any.whl

%cd /content/TensorRT-7.2.3.4/graphsurgeon/
!sudo pip3 install graphsurgeon-0.4.5-py2.py3-none-any.whl
%cd /content/TensorRT-7.2.3.4/onnx_graphsurgeon
!sudo pip3 install onnx_graphsurgeon-0.2.6-py2.py3-none-any.whl
%cd /content/

!pip install tensorflow-gpu==1.15.3
!pip install tf_slim==1.0.0

!git clone https://github.com/nolanliou/mobile-deeplab-v3-plus
!pip install superannotate
!pip install google-resumable-media==0.5.0

!sudo apt-get update
!sudo apt-get install libhdf5-serial-dev hdf5-tools libhdf5-dev zlib1g-dev zip libjpeg8-dev liblapack-dev libblas-dev gfortran
!sudo apt-get install python3-pip
!pip3 install -U pip testresources setuptools==49.6.0
!pip3 install future==0.18.2 mock==3.0.5 h5py==2.10.0 keras_preprocessing==1.1.1 keras_applications==1.0.8 gast==0.2.2 futures protobuf pybind11
!pip3 install --pre --extra-index-url https://developer.download.nvidia.com/compute/redist/jp/v43 'tensorflow<2'
!pip3 install pycuda

# 2. Setup your SuperAnnotate token, project names of trianing images and desired output model name

In [ ]:
TOKEN = "For Pro Users, the Token-ID can be generated in the teams section"
PROJECT_NAME = "City"
OUTPUT_MODEL = "sa-tutorial"

# 3. Download and preprocess the data for training

In [ ]:
import superannotate as sa
import json
import os 
from shutil import copy, make_archive
import re

os.environ['CUDA_INSTALL_DIR']= "/usr/local/cuda/"
os.environ['CUDNN_INSTALL_DIR']= "/usr/local/cuda/"
os.environ['LD_LIBRARY_PATH']+= ":/usr/lib64-nvidia/:/content/TensorRT-7.2.3.4/include/:/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/:/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/bin/"
os.environ['PATH']+= ":/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib:/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/bin/"
!sudo rm /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/libnvinfer.so.7
!sudo ln -s /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/libnvinfer.so.7.2.3 /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/libnvinfer.so.7
!sudo cp -r /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/*.* /usr/lib/ 
!sudo cp -r /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/bin/*.* /usr/bin/ 

proj_folder = dict(zip(PROJECT_NAME, ["proj_" + str(i) for i in range(len(PROJECT_NAME))]))
token_json = {"token": TOKEN}
with open('sa_config.json', 'w') as f:
  json.dump(token_json, f)

sa.init('sa_config.json')

dataset_base_dir = '/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg'
if not os.path.exists(dataset_base_dir):
  os.mkdir(dataset_base_dir)

export = sa.prepare_export(PROJECT_NAME, annotation_statuses=["Completed"], include_fuse=True)
sa.download_export(PROJECT_NAME, export, dataset_base_dir)

if not os.path.exists(dataset_base_dir + '/VOC2012'):
  os.makedirs(dataset_base_dir + '/VOC2012/JPEGImages')
  os.makedirs(dataset_base_dir + '/VOC2012/SegmentationClass')
  os.makedirs(dataset_base_dir + '/VOC2012/ImageSets/Segmentation')
  os.makedirs(dataset_base_dir + '/tfrecord')

%cd /content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/
!rm -f *.json
!rm -f *___save.png

folder_content = os.listdir('.')
for entry in folder_content:
  if len(entry.split('.'))==1:
    continue
  if entry.endswith('___fuse.png'):
    im_name, _ = os.path.splitext(entry[:-11])
    copy(entry, 'VOC2012/SegmentationClass/' + im_name + '.png')
  else:
    copy(entry, 'VOC2012/JPEGImages/' + entry)

with open(dataset_base_dir + '/VOC2012/ImageSets/Segmentation/trainval.txt', 'w') as f:
  image_names = os.listdir(dataset_base_dir + '/VOC2012/JPEGImages')
  for image_name in image_names:
    file_name, _ = os.path.splitext(image_name)
    f.write(file_name + '\n')
  
data_size = len(image_names)
val_size = int(data_size * 0.1)
train_size = data_size - val_size
!shuf VOC2012/ImageSets/Segmentation/trainval.txt | tee >(head -n "{val_size}" > VOC2012/ImageSets/Segmentation/val.txt) | tail -n "{train_size}" > VOC2012/ImageSets/Segmentation/train.txt
!python ../remove_gt_colormap.py --original_gt_folder="VOC2012/SegmentationClass" --output_dir="VOC2012/SegmentationClassRaw"

%cd /content/mobile-deeplab-v3-plus/datasets/

with open('build_voc2012_data.py') as f:
  record_gen = f.read()

record_gen = re.sub(r"import pdb\; pdb\.set\_trace\(\)\n  ", "", record_gen)

with open('build_voc2012_data.py', 'w') as f:
  f.write(record_gen)

!python build_voc2012_data.py \
  --image_folder="pascal_voc_seg/VOC2012/JPEGImages" \
  --semantic_segmentation_folder="pascal_voc_seg/VOC2012/SegmentationClassRaw" \
  --list_folder="pascal_voc_seg/VOC2012/ImageSets/Segmentation" \
  --image_format="jpg" \
  --output_dir="pascal_voc_seg/tfrecord/"

%cd /content/mobile-deeplab-v3-plus/
objdata = 'segmentation_dataset.py'
with open(objdata) as f:
    s = f.read()

with open('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/classes/classes.json', 'r') as f:
  class_data = json.load(f)

num_classes = len(class_data) + 1
s = re.sub('num_classes=21','num_classes=' + str(num_classes),s)
s = re.sub("'train': 1464","'train': " + str(train_size),s)
s = re.sub("'trainval': 2913","'trainval': " + str(data_size),s)
s = re.sub("'val': 1449","'val': " + str(val_size),s)

with open(objdata, 'w') as f:
  f.write(s)

# 4. Train the model

In [ ]:
%env PYTHONPATH=/content/mobile-deeplab-v3-plus:/env/python

import tensorflow as tf
from mobilenet_v2 import MobilenetV2
import urllib

%cd /content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/
if not os.path.exists('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2'):
  os.mkdir('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2')

%cd mnv2 
!wget https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz
!tar xvf mobilenet_v2_1.0_224.tgz

if not os.path.exists('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2_adapted'):
  os.mkdir('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2_adapted')

%cd /content/mobile-deeplab-v3-plus/

tf.reset_default_graph()

# For simplicity we just decode jpeg inside tensorflow.
# But one can provide any input obviously.
file_input = tf.placeholder(tf.string, ())

image = tf.image.decode_jpeg(tf.read_file(file_input))

images = tf.expand_dims(image, 0)
images = tf.cast(images, tf.float32) / 128. - 1
images.set_shape((None, None, None, 3))
images = tf.image.resize_images(images, (224, 224))
mobilenet_model = MobilenetV2()

logits, endpoints = mobilenet_model.forward(images, is_training=False)

exclude = ['global_step']
variables_to_restore = tf.contrib.slim.get_variables_to_restore(exclude=exclude)
variables_map = {}
for v in variables_to_restore:
  old_name = v.name.split(':')[0]
  old_name = old_name.replace('conv2d/kernel', 'weights')
  old_name = old_name.replace('conv2d/bias', 'biases')
  variables_map[old_name] = v
tf.train.init_from_checkpoint('/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2/mobilenet_v2_1.0_224.ckpt', variables_map)
filename, _ = urllib.request.urlretrieve('https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG')

saver = tf.train.Saver()
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  x = endpoints['Predictions'].eval(feed_dict={file_input: filename})
  saver.save(sess, '/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2_adapted/mobilenet-v2-224.ckpt')

if not os.path.exists("/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/checkpoint"):
  os.mkdir("/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/checkpoint")

train_iters = 15000
%cd /content/mobile-deeplab-v3-plus/
!python run.py --dataset_dir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/tfrecord"\
  --dataset_name="pascal_voc2012"\
  --logdir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/checkpoint"\
  --model_type="deeplab-v3-plus"\
  --train_subset="train"\
  --base_learning_rate=0.001\
  --num_clones=1\
  --model_input_size=256\
  --model_input_size=256\
  --training_number_of_steps="{train_iters}"\
  --pretrained_backbone_model_dir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/mnv2_adapted"

# 5. Generate and download the files needed for the OAK-D device

In [ ]:
import requests
from google.colab import files
from tensorflow.python.compiler.tensorrt import trt_convert as trt

if not os.path.exists('/content/export'):
  os.mkdir('/content/export')

!python run.py --dataset_dir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/tfrecord"\
  --dataset_name="pascal_voc2012"\
  --logdir="/content/mobile-deeplab-v3-plus/datasets/pascal_voc_seg/checkpoint"\
  --model_type="deeplab-v3-plus"\
  --model_input_size=256\
  --model_input_size=256\
  --mode=export\
  --export_dir="/content/export"

save_model_dir = "/content/export/" + os.listdir("/content/export")[0]

converter = trt.TrtGraphConverter(input_saved_model_dir="/content/export/1623136798", precision_mode="FP16")
converter.convert()
converter.save("/content/trt")

make_archive(OUTPUT_MODEL, 'zip', "/content/trt")
files.download(OUTPUT_MODEL + '.zip')

# 6. Test the custom model


Afterwards you can load the files in the archive with tf-trt on jetson devices and integrate it to your pipelines.